# Makam pitch sequence classification with LSTM

### Library importing for file reading and preprocessing

### Preprocessing pitch files

Before proceeding, the pitch files on the CompMusic Dunya makam corpus need to be converted in the quantized pitch series encoding constructed as described in the pseudocode below.

In [1]:
import glob
import os
import numpy as np
from keras.preprocessing import sequence
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


### File reading, octave folded encoding

In [55]:
#quantized octave-folded pitch file directory
ofq_read_dir = "./qdata/" 

#Makam list for more efficient file searching during label retrieval
makams = ["Acemasiran", "Acemkurdi", "Bestenigar", "Beyati", "Hicaz", "Hicazkar", "Huseyni", "Huzzam", "Karcigar", "Kurdilihicazkar", "Mahur", "Muhayyer", "Neva", "Nihavent", "Rast", "Saba", "Segah", "Sultaniyegah", "Suzinak", "Ussak"]

all_qpitch = [] #array holding all quantized pitch series as strings
y = [] #holds makam labels
max_length = 0
for root, dirs, files in os.walk(ofq_read_dir):
    for name in files:
        if '.pitch' in name:
            #retrieve label from parent of original path
            for makam in makams:
                if (os.path.isfile("./otmm_makam_recognition_dataset/data/" + makam + "/" + name) == True):
                    y.append(makams.index(makam))
                    break
            
            with open(os.path.join(root, name)) as f:
                content = f.read()
                all_qpitch.append(content)
print("File number:", len(y))
print("Array length:", len(all_qpitch))

File number: 1000
Array length: 1000


### Preprocessing
Pading input sequences

In [56]:
#Tokenization
print(all_qpitch[0][0:100])
top_k = 500
tokenizer = Tokenizer(num_words=top_k, filters='', split='\n')
tokenizer.fit_on_texts(all_qpitch)
seqs = tokenizer.texts_to_sequences(all_qpitch)
print(seqs[0][0:100])


198.1,2
200.8,1
198.1,1
200.8,1
203.4,1
206.1,1
208.8,1
211.5,1
208.8,1
206.1,1
203.4,1
208.8,1
206.
[129, 23, 5, 23, 43, 48, 49, 42, 49, 48, 43, 49, 48, 43, 48, 43, 23, 5, 2, 5, 23, 5, 2, 129, 2, 5, 2, 129, 2, 129, 2, 5, 2, 129, 2, 5, 2, 5, 23, 5, 23, 5, 2, 5, 23, 28, 51, 170, 51, 28, 51, 54, 46, 33, 46, 54, 51, 28, 21, 28, 51, 54, 51, 28, 51, 28, 51, 28, 51, 28, 51, 170, 21, 28, 51, 62, 71, 64, 67, 70, 73, 72, 66, 59, 66, 72, 73, 70, 67, 70, 73, 72, 66, 72, 73, 72, 66, 59, 56, 53]


In [57]:
#Padding
X = sequence.pad_sequences(np.asarray(seqs), maxlen = 10000, padding='post')
#print(X)
print(X[1][0:100])

[ 34  44  41  44  34  44  34  44  10  17  41  44  34  44  17  41  45  38
  45  38 198  38 198  38 198  29  33  46  54  51  28  21  37  47  19  14
  24  61  76  80  82  84  78  55  50  68  89  94  89  68  50  84  82  24
  19  21  12   2   5  23  43  48  49  42  26   8   1   3  18  31 150  34
  44 160 150  34  44  34  44  41  17  10  11  16  13   6  13  16 154  16
  13  16  11 157 135  16  13   6   9  22]


### Library importing for deep learning

In [58]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

### Train - Test split

In [59]:
y_c = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, np.asarray(y_c), test_size=0.33, random_state=41)

In [62]:
print(X_test.shape)

(330, 10000)


### Building the LSTM

In [73]:
#Network topology
model = Sequential()

#model.add(Embedding(top_k, embedding_vector_len, input_length=len(X[0])))
model.add(Embedding(500, 64, input_length=10000)) #smaller input length for testing
#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(70))
model.add(Dropout(0.2))
model.add(Dense(20, activation='sigmoid'))

In [74]:
#Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], )
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 10000, 64)         32000     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 10000, 32)         6176      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 5000, 32)          0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 5000, 32)          0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 70)                28840     
_________________________________________________________________
dropout_24 (Dropout)         (None, 70)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 20)              

In [75]:
#Fitting
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1, validation_split=0.33)

/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 448 samples, validate on 222 samples
Epoch 1/10
448/448 [==============================] - 34s 76ms/step - loss: 0.6836 - accuracy: 0.7235 - val_loss: 0.6666 - val_accuracy: 0.8809
Epoch 2/10
448/448 [==============================] - 35s 77ms/step - loss: 0.6267 - accuracy: 0.8603 - val_loss: 0.5194 - val_accuracy: 0.8590
Epoch 3/10
448/448 [==============================] - 44s 99ms/step - loss: 0.4218 - accuracy: 0.8987 - val_loss: 0.3088 - val_accuracy: 0.9500
Epoch 4/10
448/448 [==============================] - 41s 91ms/step - loss: 0.2821 - accuracy: 0.9448 - val_loss: 0.2325 - val_accuracy: 0.9500
Epoch 5/10
448/448 [==============================] - 45s 101ms/step - loss: 0.2277 - accuracy: 0.9497 - val_loss: 0.2070 - val_accuracy: 0.9500
Epoch 6/10
448/448 [==============================] - 43s 96ms/step - loss: 0.2100 - accuracy: 0.9500 - val_loss: 0.2023 - val_accuracy: 0.9500
Epoch 7/10
448/448 [==============================] - 42s 94ms/step - loss: 0.2086 - accu

In [45]:
#Model evaluation
scores = model.evaluate(X_test, y_test, verbose=1)
print(scores)

330/330 [==============================] - 1s 3ms/step
[0.19809791626352252, 0.9499999284744263]
